# 05. A/B 테스트 분석 실습

SQL로 A/B 테스트 결과를 분석해봅니다.

**시나리오:**
> 새로운 모바일 UI를 테스트하고 있습니다.
> - 실험군 (Treatment): mobile 사용자 - 새 UI
> - 대조군 (Control): desktop 사용자 - 기존 UI
> - 목표: 구매 전환율 비교

**규칙:**
- 힌트는 최대한 안 보고 먼저 시도하세요
- 막히면 힌트를 펼쳐보세요
- 완료 후 `solution.sql`과 비교하세요

## 환경 설정

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats

# 데이터베이스 연결
DB_PATH = Path("../data/crm.db")
conn = sqlite3.connect(DB_PATH)

# SQL 실행 헬퍼 함수
def sql(query):
    """SQL 쿼리 실행 및 결과 반환"""
    return pd.read_sql(query, conn)

print("데이터베이스 연결 완료!")

---
## 준비: 실험 데이터 파악

In [ ]:
# 디바이스별 사용자 분포 확인 (실험 그룹)
sql("""
SELECT 
    device,
    COUNT(DISTINCT user_id) as unique_users,
    COUNT(*) as total_events
FROM events
GROUP BY device
""")

In [ ]:
# 이벤트 유형별 분포 확인
sql("""
SELECT 
    event_type,
    COUNT(*) as count
FROM events
GROUP BY event_type
ORDER BY count DESC
""")

---
# Mission 1: 실험 그룹별 기본 지표

대조군과 실험군의 기본 지표를 비교해봅시다.

## Mission 1-1: 그룹별 사용자 수 및 전환 수

**문제:** 각 그룹(mobile vs desktop)별로 다음을 계산하세요:
- 총 사용자 수 (page_view 기준)
- 구매 완료 사용자 수 (purchase 기준)

출력 컬럼: `experiment_group`, `total_users`, `converted_users`

In [ ]:
# TODO: 그룹별 사용자 수와 전환 수를 계산하는 SQL을 작성하세요
sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
)
SELECT 
    v.device as experiment_group,
    COUNT(DISTINCT v.user_id) as total_users,
    _____ as converted_users
FROM visitors v
LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
GROUP BY v.device
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 전환 사용자 수: `COUNT(DISTINCT c.user_id)`
- LEFT JOIN을 사용하여 전환하지 않은 사용자도 포함

</details>

## Mission 1-2: 전환율 계산

**문제:** 각 그룹의 전환율(Conversion Rate)을 계산하세요.

```
전환율 = 전환 사용자 수 / 총 사용자 수 × 100
```

출력 컬럼: `experiment_group`, `total_users`, `converted_users`, `conversion_rate`

In [ ]:
# TODO: 전환율을 계산하는 SQL을 작성하세요
sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
),
group_metrics AS (
    SELECT 
        v.device as experiment_group,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users
    FROM visitors v
    LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
    GROUP BY v.device
)
SELECT 
    experiment_group,
    total_users,
    converted_users,
    ROUND(_____ * 100, 2) as conversion_rate
FROM group_metrics
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 전환율: `converted_users * 1.0 / total_users`

</details>

---
# Mission 2: 전환율 차이 분석

두 그룹 간의 차이를 분석해봅시다.

## Mission 2-1: 전환율 차이 계산

**문제:** 실험군(mobile)과 대조군(desktop)의 전환율 차이를 계산하세요.

```
절대 차이 = 실험군 전환율 - 대조군 전환율
상대 차이(Lift) = (실험군 - 대조군) / 대조군 × 100
```

출력: 각 그룹의 전환율, 절대 차이, 상대적 개선율(Lift)

In [ ]:
# TODO: 전환율 차이를 계산하는 SQL을 작성하세요
sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
),
group_metrics AS (
    SELECT 
        v.device as experiment_group,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users,
        COUNT(DISTINCT c.user_id) * 100.0 / COUNT(DISTINCT v.user_id) as conversion_rate
    FROM visitors v
    LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
    GROUP BY v.device
),
comparison AS (
    SELECT 
        MAX(CASE WHEN experiment_group = 'desktop' THEN conversion_rate END) as control_rate,
        MAX(CASE WHEN experiment_group = 'mobile' THEN conversion_rate END) as treatment_rate
    FROM group_metrics
)
SELECT 
    ROUND(control_rate, 2) as control_conversion_rate,
    ROUND(treatment_rate, 2) as treatment_conversion_rate,
    ROUND(treatment_rate - control_rate, 2) as absolute_difference,
    ROUND(_____, 2) as relative_lift_percent
FROM comparison
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- Lift: `(treatment_rate - control_rate) * 100.0 / control_rate`

</details>

## Mission 2-2: 통계적 유의성 검정 데이터 준비

**문제:** 통계 검정에 필요한 데이터를 추출하세요.

출력: 각 그룹의 `n` (표본 크기), `x` (성공 수), `p` (전환율)

In [ ]:
# 통계 검정용 데이터 추출
test_data = sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
)
SELECT 
    v.device as experiment_group,
    COUNT(DISTINCT v.user_id) as n,
    COUNT(DISTINCT c.user_id) as x,
    COUNT(DISTINCT c.user_id) * 1.0 / COUNT(DISTINCT v.user_id) as p
FROM visitors v
LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
GROUP BY v.device
""")

test_data

## Mission 2-3: Python으로 통계 검정 수행

**문제:** 위에서 추출한 데이터로 카이제곱 검정을 수행하세요.

귀무가설(H0): 두 그룹의 전환율에 차이가 없다
대립가설(H1): 두 그룹의 전환율에 차이가 있다

In [ ]:
# TODO: 카이제곱 검정을 수행하세요

# 데이터 추출
control = test_data[test_data['experiment_group'] == 'desktop'].iloc[0]
treatment = test_data[test_data['experiment_group'] == 'mobile'].iloc[0]

# 분할표 생성
# [[control_converted, control_not_converted],
#  [treatment_converted, treatment_not_converted]]
contingency_table = [
    [int(control['x']), int(control['n'] - control['x'])],
    [int(treatment['x']), int(treatment['n'] - treatment['x'])]
]

print("분할표 (Contingency Table):")
print(f"Control:   Converted={contingency_table[0][0]}, Not Converted={contingency_table[0][1]}")
print(f"Treatment: Converted={contingency_table[1][0]}, Not Converted={contingency_table[1][1]}")
print()

# 카이제곱 검정 수행
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")
print()

# 결과 해석
alpha = 0.05
if p_value < alpha:
    print(f"p-value ({p_value:.4f}) < {alpha}")
    print("결론: 통계적으로 유의합니다. 귀무가설을 기각합니다.")
    print("두 그룹의 전환율에 유의미한 차이가 있습니다.")
else:
    print(f"p-value ({p_value:.4f}) >= {alpha}")
    print("결론: 통계적으로 유의하지 않습니다. 귀무가설을 기각할 수 없습니다.")
    print("두 그룹의 전환율 차이가 우연에 의한 것일 수 있습니다.")

---
# Mission 3: 세부 분석

더 깊이 분석해봅시다.

## Mission 3-1: 채널별 전환율 비교

**문제:** 유입 채널별로 두 그룹의 전환율을 비교하세요.

출력 컬럼: `channel`, `control_rate`, `treatment_rate`, `difference`

In [ ]:
# TODO: 채널별 전환율을 비교하는 SQL을 작성하세요
sql("""
WITH visitors AS (
    SELECT DISTINCT device, channel, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, channel, user_id
    FROM events
    WHERE event_type = 'purchase'
),
channel_metrics AS (
    SELECT 
        v.channel,
        v.device,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users,
        COUNT(DISTINCT c.user_id) * 100.0 / COUNT(DISTINCT v.user_id) as conversion_rate
    FROM visitors v
    LEFT JOIN converters c ON v.device = c.device AND v.channel = c.channel AND v.user_id = c.user_id
    GROUP BY v.channel, v.device
)
SELECT 
    channel,
    ROUND(MAX(CASE WHEN device = 'desktop' THEN conversion_rate END), 2) as control_rate,
    ROUND(MAX(CASE WHEN device = 'mobile' THEN conversion_rate END), 2) as treatment_rate,
    ROUND(
        MAX(CASE WHEN device = 'mobile' THEN conversion_rate END) -
        MAX(CASE WHEN device = 'desktop' THEN conversion_rate END),
        2
    ) as difference
FROM channel_metrics
GROUP BY channel
ORDER BY difference DESC
""")

## Mission 3-2: 일별 전환율 추이

**문제:** 실험 기간 동안 일별 전환율 추이를 분석하세요.

출력 컬럼: `date`, `control_rate`, `treatment_rate`

In [ ]:
# TODO: 일별 전환율 추이를 분석하는 SQL을 작성하세요
sql("""
WITH daily_visitors AS (
    SELECT DISTINCT device, event_date, user_id
    FROM events
    WHERE event_type = 'page_view'
),
daily_converters AS (
    SELECT DISTINCT device, event_date, user_id
    FROM events
    WHERE event_type = 'purchase'
),
daily_metrics AS (
    SELECT 
        v.event_date,
        v.device,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users,
        COUNT(DISTINCT c.user_id) * 100.0 / NULLIF(COUNT(DISTINCT v.user_id), 0) as conversion_rate
    FROM daily_visitors v
    LEFT JOIN daily_converters c 
        ON v.device = c.device AND v.event_date = c.event_date AND v.user_id = c.user_id
    GROUP BY v.event_date, v.device
)
SELECT 
    event_date as date,
    ROUND(MAX(CASE WHEN device = 'desktop' THEN conversion_rate END), 2) as control_rate,
    ROUND(MAX(CASE WHEN device = 'mobile' THEN conversion_rate END), 2) as treatment_rate
FROM daily_metrics
GROUP BY event_date
ORDER BY event_date
LIMIT 14
""")

---
# Mission 4: 비즈니스 영향 분석

실험 결과의 비즈니스 영향을 추정해봅시다.

## Mission 4-1: 예상 추가 전환 수 계산

**문제:** 실험군(mobile)에 대조군과 같은 전환율을 적용했다면,
얼마나 적은/많은 전환이 발생했을지 계산하세요.

```
추가 전환 수 = 실험군 실제 전환 - (실험군 사용자 수 × 대조군 전환율)
```

In [ ]:
# TODO: 예상 추가 전환 수를 계산하는 SQL을 작성하세요
sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
),
group_metrics AS (
    SELECT 
        v.device as experiment_group,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users,
        COUNT(DISTINCT c.user_id) * 1.0 / COUNT(DISTINCT v.user_id) as conversion_rate
    FROM visitors v
    LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
    GROUP BY v.device
)
SELECT 
    treatment.total_users as treatment_users,
    treatment.converted_users as actual_conversions,
    ROUND(treatment.total_users * control.conversion_rate, 0) as expected_conversions_at_control_rate,
    treatment.converted_users - ROUND(treatment.total_users * control.conversion_rate, 0) as additional_conversions
FROM group_metrics treatment, group_metrics control
WHERE treatment.experiment_group = 'mobile'
AND control.experiment_group = 'desktop'
""")

## Mission 4-2: 의사결정 권고

**문제:** 실험 결과를 종합하여 의사결정 권고안을 작성하세요.

In [ ]:
# 종합 실험 결과 요약
summary = sql("""
WITH visitors AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'page_view'
),
converters AS (
    SELECT DISTINCT device, user_id
    FROM events
    WHERE event_type = 'purchase'
),
group_metrics AS (
    SELECT 
        v.device as experiment_group,
        COUNT(DISTINCT v.user_id) as total_users,
        COUNT(DISTINCT c.user_id) as converted_users,
        COUNT(DISTINCT c.user_id) * 100.0 / COUNT(DISTINCT v.user_id) as conversion_rate
    FROM visitors v
    LEFT JOIN converters c ON v.device = c.device AND v.user_id = c.user_id
    GROUP BY v.device
)
SELECT 
    experiment_group,
    total_users,
    converted_users,
    ROUND(conversion_rate, 2) as conversion_rate_pct
FROM group_metrics
ORDER BY experiment_group
""")

print("=" * 50)
print("실험 결과 요약")
print("=" * 50)
print(summary.to_string(index=False))
print()

control_rate = summary[summary['experiment_group']=='desktop']['conversion_rate_pct'].values[0]
treatment_rate = summary[summary['experiment_group']=='mobile']['conversion_rate_pct'].values[0]
lift = (treatment_rate - control_rate) / control_rate * 100

print(f"전환율 차이: {treatment_rate - control_rate:.2f}%p")
print(f"상대적 개선(Lift): {lift:.1f}%")
print(f"통계적 유의성: p-value = {p_value:.4f}")
print()

print("=" * 50)
print("의사결정 권고")
print("=" * 50)
if p_value < 0.05 and treatment_rate > control_rate:
    print("RECOMMENDATION: 새 UI를 전체 사용자에게 적용하세요.")
    print(f"- 전환율이 {lift:.1f}% 개선될 것으로 예상됩니다.")
elif p_value < 0.05 and treatment_rate < control_rate:
    print("RECOMMENDATION: 새 UI 적용을 중단하세요.")
    print(f"- 전환율이 {abs(lift):.1f}% 하락할 것으로 예상됩니다.")
else:
    print("RECOMMENDATION: 추가 실험이 필요합니다.")
    print("- 더 많은 표본을 수집하거나 실험 기간을 연장하세요.")

---
# Mission 5: 실무 시나리오

**시나리오:**

> 당신은 이 A/B 테스트 결과를 경영진에게 보고해야 합니다.
> 다음 내용을 포함하는 보고서를 작성하세요:
>
> 1. 실험 개요 (목적, 가설, 기간)
> 2. 주요 결과 (전환율, 차이, 유의성)
> 3. 비즈니스 영향 (예상 추가 매출 등)
> 4. 권고사항

**당신의 분석을 아래에 작성하세요:**

### A/B 테스트 결과 보고서

(여기에 보고서를 작성하세요)

**1. 실험 개요**
- 목적: 
- 가설: 
- 기간: 

**2. 주요 결과**
- 대조군 전환율: 
- 실험군 전환율: 
- 차이: 
- 통계적 유의성: 

**3. 비즈니스 영향**
- 

**4. 권고사항**
- 

---
# 회고

실습을 마치며 아래 질문에 답해보세요.

1. **어떤 부분이 어려웠나요?**

2. **통계적 유의성과 실무적 유의성의 차이는?**

3. **A/B 테스트를 실무에서 어떻게 활용하겠습니까?**

4. **면접에서 이 내용을 어떻게 설명하겠습니까?**

In [ ]:
# 데이터베이스 연결 종료
conn.close()
print("실습 완료!")